### Chapter 1

- Using the bytearray() function to convert the text to IDs

In [1]:
example_text = "This is an example text"
barray = bytearray(example_text, encoding="utf-8")
#The example_text is converted to bytearray; Now if we return the bytearray as a list we will get the bytes number for each text
example_text_byte_ids = list(barray)
print("The example_text is successfully converted to an array having numbers for each text:")
print(example_text_byte_ids)



The example_text is successfully converted to an array having numbers for each text:
[84, 104, 105, 115, 32, 105, 115, 32, 97, 110, 32, 101, 120, 97, 109, 112, 108, 101, 32, 116, 101, 120, 116]


- ##### Drawback of this approach
-       Ids is generated for every character in the text

In [18]:
print("Length of input text:",len(example_text))
print("Length of IDs:", len(example_text_byte_ids))

Length of input text: 23
Length of IDs: 23


- The first 256-values of BPE tokenizer

- The first 256-values of BPE tokenizer is single-character tokens

In [ ]:
import tiktoken

tokenizer = tiktoken.get_encoding('gpt2')

#Print the first 300 tokens and their respective characters 
for i in range(1, 300):
    decoded = tokenizer.decode([i])
    print(f"\n{i}:{decoded}")


1:"

2:#

3:$

4:%

5:&

6:'

7:(

8:)

9:*

10:+

11:,

12:-

13:.

14:/

15:0

16:1

17:2

18:3

19:4

20:5

21:6

22:7

23:8

24:9

25::

26:;

27:<

28:=

29:>

30:?

31:@

32:A

33:B

34:C

35:D

36:E

37:F

38:G

39:H

40:I

41:J

42:K

43:L

44:M

45:N

46:O

47:P

48:Q

49:R

50:S

51:T

52:U

53:V

54:W

55:X

56:Y

57:Z

58:[

59:\

60:]

61:^

62:_

63:`

64:a

65:b

66:c

67:d

68:e

69:f

70:g

71:h

72:i

73:j

74:k

75:l

76:m

77:n

78:o

79:p

80:q

81:r

82:s

83:t

84:u

85:v

86:w

87:x

88:y

89:z

90:{

91:|

92:}

93:~

94:�

95:�

96:�

97:�

98:�

99:�

100:�

101:�

102:�

103:�

104:�

105:�

106:�

107:�

108:�

109:�

110:�

111:�

112:�

113:�

114:�

115:�

116:�

117:�

118:�

119:�

120:�

121:�

122:�

123:�

124:�

125:�

126:�

127:�

128:�

129:�

130:�

131:�

132:�

133:�

134:�

135:�

136:�

137:�

138:�

139:�

140:�

141:�

142:�

143:�

144:�

145:�

146:�

147:�

148:�

149:�

150:�

151:�

152:�

153:�

154:�

155:�

156:�

157:�

158:�

1

## Main BPE ALgorithm

- Split the input text into individual bytes
- Repeatedly find & replace (merge) adjacent tokens (pairs) when they match any pair in the learned BPE merges (from highest to lowest “rank,” i.e., in the order they were learned)
- Continue merging until no more merges can be applied
- The final list of token IDs is the encoded output

In [9]:
from collections import Counter, deque
from functools import lru_cache
import json

##### Example of preprocessing the text in train_function

In [7]:
text = "Hello World"
example_preprocessed = []
for i, char in enumerate(text):
    if char == " " and i!= 0 :
        example_preprocessed.append("_")
    if char != " ":
        example_preprocessed.append(char)
        
example_preprocessed = "".join(example_preprocessed)
print("The space between the text is replaced with a special character:-", example_preprocessed)


The space between the text is replaced with a special character:- Hello_World


## BPE Class

In [ ]:
from tqdm import tqdm

class BPEtokenizer:
    def __init__(self):
        
        #Maps the token_id with token_str (e.g. {11456:"some"}) 
        self.vocab = {}
        
        #Maps token_str to token_id (e.g. {"some":11456})
        self.inverse_vocab = {}
        
        #Dictionary containing all the merge_rules
        self.bpe_merges = {}
        
        
    def train(self, text, vocab_size, allowed_specials = {"<|endoftext|>"}):
        #========We replace the space present between text with '_' inorder to preserve the word boundaries=============
        processed_text = []
        for i, char in enumerate(text):
            if char == " " and i != 0:
                processed_text.append("_")
            if char != " ":
                processed_text.append(char)
                
        processed_text = "".join(processed_text) 
        
        
        #==========Initialize the Vocabulary with unique characters==========
        #Get the first 256 ASCII characters
        unique_chars = [ chr(i) for i in range(256)]
        
        #Now add the unique character which are in processed_text but not in unique_chars list
        unique_chars.extend(char for char in sorted(set(processed_text)) if char not in unique_chars)
        
        #=================Create the Vocab & Inverse_Vocab Dictionaries=================
        self.vocab = {i : char for i,char in enumerate(unique_chars)}
        self.inverse_vocab = {char : i for i, char in self.vocab.items()}
        
        #================Add allowed special tokens=============
        if allowed_specials:
            for token in allowed_specials:
                if token not in self.inverse_vocab:
                    new_id = len(self.vocab)
                    self.inverse_vocab[token] = new_id
                    self.vocab[new_id] = token
                    
                    
        #================Tokenize the processed text into token_IDs================
        token_ids = [self.inverse_vocab[char] for char in processed_text]
        
        #================BPE Algorithm steps 1-3================
        for new_id in tqdm(range(len(self.vocab), vocab_size), desc="Training BPE"):
            pair_id = self.find_freq_pair(token_ids, mode= "most")
            
            if pair_id is None: #No more pair to merge; Stop the training
                break
            token_ids = self.replace_pairs(token_ids, pair_id, new_id)
            self.bpe_merges[pair_id] = new_id
        
        #================Update the vocabulary with the new merged_tokens================
        for (p0, p1), new_id in self.bpe_merges.items():
            merged_token = self.vocab[p0] + self.vocab[p1]
            self.vocab[new_id] = merged_token
            self.inverse_vocab[merged_token] = new_id
        
        
    #=======================This function is used to load pretrained-vocabularies and Merge Rules from Open-AI Gpt-2=======================
    def load_vocab_and_merges_from_OpenAI(self, vocab_path, bpe_merges_path):
        
        
        #===================Load Vocabulary===================
        with open(vocab_path, "r", encoding="utf-8") as file:
            loaded_vocab = json.load(file)
            
            #Using the loaded_vocab's vocab and inverse_vocab dictionaries
            self.vocab = {int(v):k for k,v in loaded_vocab.items()} #token_id:token_str
            self.inverse_vocab = {k: int(v) for k,v in loaded_vocab.items()} #token_str : token_id
            
        #===================Load BPE Merge Rules===================
        with open(bpe_merges_path, "r", encoding="utf-8") as file:
            lines = file.readlines()
            
            #Skip the header line if present
            if lines and lines[0].startswith("#"):
                lines = lines[1:]
                
            for rank, line in enumerate(lines):
                pair = tuple(line.strip().split())
                
                #Check if the pair contains exactly 2 entries or not
                if(len(pair) != 2): 
                    print(f"Line{rank +1} has more than 2 entries: {line.strip()}")
                    continue
                token1, token2 = pair
                
                #Check if the str_tokens we get from the pair are in inverse_vocabulary or not
                if token1 in self.inverse_vocab and token2 in self.inverse_vocab: 
                    token_id1 = self.inverse_vocab[token1]
                    token_id2 = self.inverse_vocab[token2]
                    merged_token = token1 + token2
                    
                    #Check if the string(merged_token) is in inverse_vocabulary or not
                    if merged_token in self.inverse_vocab:
                        merged_token_id = self.inverse_vocab[merged_token]
                        
                        #Updating the BPE merge rule list
                        self.bpe_merges[(token_id1, token_id2)] = merged_token_id
                        
                    else:
                        print(f"Merged token {merged_token} not found in vocabulary; ===Skip====")
                else:
                    print(f"Skipping the pair:{pair} as both of the tokens are not in vocabulary")
                    
    #========================This function is used to encode the input_text into list of token_ids at Inference========================                
    def encode(self, text:str):
        """ 
            This function will generate token_ids based on the BPE merge rules and the vocabularies it learned during the training   
        """
        tokens = []
        
        #Split the text into tokens, Keeping the newline intact
        words = text.replace("\n", " \n ").split()  #Make sure that the new_line seperator "\n" is treated as a separate token
        
        for i, word in enumerate(words):
            if i > 0 and not word.startswith("\n"): 
                tokens.append("_" + word) #Add "_" to words that are after a space or "\n"
            else:
                tokens.append(word)
                
        token_ids = []
        for token in tokens:
            #Check if the token is already present in the vocabulary or not
            if token in self.inverse_vocab:
                token_id = self.inverse_vocab[token]
                token_ids.append(token_id)
            else:
                #Do subword_tokenization using BPE
                sub_token_ids = self.tokenize_with_bpe(token)
                token_ids.extend(sub_token_ids)
                
        return token_ids
                
        
    #=======================This function is used to tokenize a sinlge token using BPE merge rules=======================                   
    def tokenize_with_bpe(self, token):
        
        #Tokenize the tokens into individual characters(it can be interpreted as initial token_Ids)
        token_ids = [self.inverse_vocab.get(char, None) for char in token]
        if None in token_ids:
            missing_chars = [char for char, tid in zip(token, token_ids) if tid is None]
            raise ValueError(f"Characters not found in vocabulary: {missing_chars}")
        
        can_merge = True
        while can_merge and len(token_ids) > 1:
            can_merge = False
            new_tokens = []
            i = 0
            while i < len(token_ids) - 1:
                pair = (token_ids[i] , token_ids[i+1])
                if pair in self.bpe_merges:
                    merged_token_id = self.bpe_merges[pair]
                    new_tokens.append(merged_token_id)
                    
                    i += 2 #Skip the next token as it is already merged
                    can_merge = True
                else:
                    new_tokens.append(token_ids[i])
                    i+=1
            if i < len(token_ids):
                new_tokens.append(token_ids[i])
            token_ids = new_tokens
                
        return token_ids
                    
        
    #======================This function is used to decode a list of token_ids back to text======================
    def decode(self, token_ids):
        decoded_string = ""
        for token_id in token_ids:
            if token_id not in self.vocab:
                raise ValueError(f"Token Id {token_id} not found in Vocabulary")
            token = self.vocab[token_id]
            if token.startswith("_"):
                #Replace the "_" with space
                decoded_string += " " + token[1:]
            else:
                decoded_string += token
            
        return decoded_string  
                
        
    #======================This function is used to save the vocabulary and BPE merges to a JSON file======================
    def save_vocab_merges(self, vocab_path, merges_path):  
        with open(vocab_path ,"w", encoding="utf-8") as file:
            json.dump({k : v for k, v in self.vocab.items()}, file, ensure_ascii=False, indent=2)
            
        #Save BPE Merges as a list of dictionaries
        with open(merges_path, "w", encoding="utf-8") as file:
            merges_list = [{"pair" : list(pair), "new_id" : new_id}
                           for pair, new_id in self.bpe_merges.items()]
            json.dump(merges_list, file, ensure_ascii=False, indent=2)
    
    #======================This function is used to Load the vocabulary and BPE merges from JSON files locally======================
    def load_vocab_merges(self, vocab_path, bpe_merges_path):
        
        #Load vocabulary
        with open(vocab_path, "r", encoding="utf-8") as file:
            loaded_vocab = json.load(file)
            self.vocab = {int(k):v for k,v in loaded_vocab.items()}
            self.inverse_vocab = {v:int(k) for k,v in loaded_vocab.items()}
            
        #Load bpe_merges
        with open(bpe_merges_path, "r", encoding="utf-8") as file:
            merges_list = json.load(file)
            for merge in merges_list:
                pair = tuple(merge["pair"])
                new_id = merge['new_id']
                self.bpe_merges[pair] = new_id
                
    
    #======================This function is used to find the most frequent pairs in the data======================
    @staticmethod
    def find_freq_pair(token_ids, mode="most"):
        pairs = Counter(zip(token_ids, token_ids[1:]))
        
        if not pairs:
            return None

        if mode == "most":
            return max(pairs.items(), key = lambda x : x[1])[0]
        elif mode == "least":
            return min(pairs.items(),key=lambda x: x [1])[0]
        else:
            return ValueError("Invalid Code: Choose mode: 'most' or 'least'")
        
    #======================This function is used to 
    @staticmethod
    def replace_pairs(token_ids, pair_id, new_id):
        dq = deque(token_ids)
        replaced = []

        while dq:
            current = dq.popleft()
            if dq and (current, dq[0]) == pair_id:
                replaced.append(new_id)
                # Remove the 2nd token of the pair, 1st was already removed
                dq.popleft()
            else:
                replaced.append(current)

        return replaced
            
            
            
        
        
        
        
        
    

#### Example of inverse_vocab and vocab

In [15]:
example_unique_chars = [chr(i) for i in range(256)]
vocab_example = {i:char for i, char in enumerate(example_unique_chars)}
inverse_vocab_example = {char:i for i,char in vocab_example.items()}

print(f"Vocabulary:-\n{vocab_example}")
print(f"Inverse Vocabulary:- \n{inverse_vocab_example}")

example_token_ids = [inverse_vocab_example[char] for char in example_preprocessed]
print(f"Example of token_ids is:-{example_token_ids}")



Vocabulary:-
{0: '\x00', 1: '\x01', 2: '\x02', 3: '\x03', 4: '\x04', 5: '\x05', 6: '\x06', 7: '\x07', 8: '\x08', 9: '\t', 10: '\n', 11: '\x0b', 12: '\x0c', 13: '\r', 14: '\x0e', 15: '\x0f', 16: '\x10', 17: '\x11', 18: '\x12', 19: '\x13', 20: '\x14', 21: '\x15', 22: '\x16', 23: '\x17', 24: '\x18', 25: '\x19', 26: '\x1a', 27: '\x1b', 28: '\x1c', 29: '\x1d', 30: '\x1e', 31: '\x1f', 32: ' ', 33: '!', 34: '"', 35: '#', 36: '$', 37: '%', 38: '&', 39: "'", 40: '(', 41: ')', 42: '*', 43: '+', 44: ',', 45: '-', 46: '.', 47: '/', 48: '0', 49: '1', 50: '2', 51: '3', 52: '4', 53: '5', 54: '6', 55: '7', 56: '8', 57: '9', 58: ':', 59: ';', 60: '<', 61: '=', 62: '>', 63: '?', 64: '@', 65: 'A', 66: 'B', 67: 'C', 68: 'D', 69: 'E', 70: 'F', 71: 'G', 72: 'H', 73: 'I', 74: 'J', 75: 'K', 76: 'L', 77: 'M', 78: 'N', 79: 'O', 80: 'P', 81: 'Q', 82: 'R', 83: 'S', 84: 'T', 85: 'U', 86: 'V', 87: 'W', 88: 'X', 89: 'Y', 90: 'Z', 91: '[', 92: '\\', 93: ']', 94: '^', 95: '_', 96: '`', 97: 'a', 98: 'b', 99: 'c', 100: 

#### Example of logic behind appending "_" in encode function

In [5]:
example_tokens = []
example_text = "Hey, I am a cat \n, I like to sleep \n"
example_words = example_text.replace("\n", " \n ").split()

for i,word in enumerate(example_words):
    if i > 0 and not word.startswith("\n"):
        example_tokens.append("_" + word)
    else:
        example_tokens.append(word)

print(example_tokens)
    

['Hey,', '_I', '_am', '_a', '_cat', '_,', '_I', '_like', '_to', '_sleep']


## Algorithm Inference

#### Getting the dataset

In [6]:
import os
import urllib.request

file_path = "./data/the-verdict.txt"

with open(file_path, "r", encoding="utf-8") as file:
    data = file.read()
    
print("="*23, f"Data successfully loaded", "="*23)
print(data[:2000])

======================= Data successfully loaded =======================
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no great surprise to me to hear that, in the height of his glory, he had dropped his painting, married a rich widow, and established himself in a villa on the Riviera. (Though I rather thought it would have been Rome or Florence.)

"The height of his glory"--that was what the women called it. I can hear Mrs. Gideon Thwing--his last Chicago sitter--deploring his unaccountable abdication. "Of course it's going to send the value of my picture 'way up; but I don't think of that, Mr. Rickham--the loss to Arrt is all I think of." The word, on Mrs. Thwing's lips, multiplied its _rs_ as though they were reflected in an endless vista of mirrors. And it was not only the Mrs. Thwings who mourned. Had not the exquisite Hermia Croft, at the last Grafton Gallery show, stopped me before Gisburn's "Moon-dancers" to say, with tears in h

In [7]:
print(f"Length of the text: {len(data)}")

Length of the text: 20479


### Initializing the tokenizer and training it

In [45]:
tokenizer = BPEtokenizer()
tokenizer.train(data, vocab_size=1000, allowed_specials={"<|endoftext|>"})


Training BPE: 100%|██████████| 743/743 [00:04<00:00, 174.74it/s]


In [33]:
print(f"Length of Vocabulary of tokenizer:- {len(tokenizer.vocab)}")

Length of Vocabulary of tokenizer:- 1000


In [34]:
print(f"Total number of merge operations performed:- {len(tokenizer.bpe_merges)}")

Total number of merge operations performed:- 743


- Encoding a random text using the encode method of our tokenizer

In [35]:
random_text = "Jack embraced beauty through art and life"
token_ids = tokenizer.encode(random_text)
print(token_ids)


[424, 95, 653, 529, 301, 310, 95, 295, 97, 466, 121, 591, 837, 116, 286, 467, 95, 325, 973]


In [36]:
print(f"Number of characters in random_text: {len(random_text)}")
print(f"Number of token Ids:- {len(token_ids)}")

Number of characters in random_text: 41
Number of token Ids:- 19


In [37]:
print(tokenizer.decode(token_ids))

Jack embraced beauty through art and life


In [39]:
for token_id in token_ids:
    print(f"{token_id}-->{tokenizer.decode([token_id])}")

424-->Jack
95--> 
653-->em
529-->br
301-->ac
310-->ed
95--> 
295-->be
97-->a
466-->ut
121-->y
591--> through
837--> ar
116-->t
286--> a
467-->nd
95--> 
325-->li
973-->fe


- A 41 character sequence is encoded into 20 token Ids, effectively cutting the input length roughly in half compared to character-byte-based encoding

## Saving & Loading the tokenizer

In [46]:
#Saving the trained tokenizer
tokenizer.save_vocab_merges(vocab_path="./save_files/vocab.json", merges_path="./save_files/merges.json")


In [50]:
#Loading the trained tokenizer
tokenizer2 = BPEtokenizer()

tokenizer2.load_vocab_merges(vocab_path="./save_files/vocab.json",bpe_merges_path="./save_files/merges.json")
print("="*23, "Tokenizer loaded successfully", "="*23)

======================= Tokenizer loaded successfully =======================


In [52]:
example_text = "Jack and Jill went up a hill and fell down."
token_ids2 = tokenizer2.encode(example_text)
print(token_ids2)

[424, 286, 467, 95, 74, 554, 108, 95, 471, 110, 116, 942, 286, 95, 276, 352, 286, 467, 464, 391, 95, 681, 119, 110, 46]
